<a href="https://colab.research.google.com/github/jpantojaj/Credit_Scoring_Specialization/blob/main/Taller_MLOps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Registro de modelos con MlFlow**

In [ ]:
!pip install mlflow --quiet

In [ ]:
import mlflow
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import warnings

In [ ]:
# informacion del modelo
model_name = "modelo_clase"

In [ ]:
# preprocesamiento de datos
X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# registrar experimento
experiment_info = mlflow.set_experiment(model_name)
experiment_id = int(dict(experiment_info)["experiment_id"])

In [ ]:
# registro del modelo con mlflow
with mlflow.start_run(experiment_id=experiment_id):
    # estimacion del modelo
    model_rf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
    model_rf.fit(X_train, y_train)

    # parametros del modelo
    params = model_rf.get_params()

    # metricas del modelo
    y_pred_train = model_rf.predict(X_train)
    y_pred_test = model_rf.predict(X_test)
    metricas = {"accuracy_train": accuracy_score(y_train, y_pred_train),
                "accuracy_test": accuracy_score(y_test, y_pred_test)}

    # log parametros del modelo
    mlflow.log_params(params)

    # log metricas
    mlflow.log_metrics(metricas)

    # log firma de input y outpur
    signature = infer_signature(X_train, model_rf.predict(X_train))

    # registro del modelo
    model_info = mlflow.sklearn.log_model(model_rf,
                                          artifact_path=model_name,
                                          registered_model_name=model_name,
                                          signature=signature)

    # informacion del modelo
    print(model_info.model_uri)

In [ ]:
# ver modelos registrados y versiones
client = MlflowClient()

# lista de modelos registrados
print("Lista de todos los modelo registrados:")
print("=" * 80)
for model in client.search_registered_models():
  print(model)
print()

# lista de modelos y versiones
print(f"Lista de versiones del modelo {model_name}:")
print("=" * 80)
for version in client.search_model_versions(f"name='{model_name}'"):
  print(version)

## **2. Mostrar información registrada del modelo**

In [ ]:
# obtener informacion del modelo registrado
mlflow.search_runs(experiment_ids=[str(experiment_id)])

## **3. Carga de modelo registrado para predicciones**

In [ ]:
# obtener ultima version del modelo
model_version = client.get_latest_versions(name=model_name)[0].version

# cargar el modelo registrado
loaded_model = mlflow.sklearn.load_model(model_uri=f"models:/{model_name}/{model_version}")

In [ ]:
# hacer predicciones con modelo registrado
X, y = datasets.load_iris(return_X_y=True)
y_pred = loaded_model.predict(X)
print(y_pred)